
Notebook Name : 04-Transform-DataLake-Geocoding

GeoLocation JSON Source File Path : "abfss://bronze@datalakestorageaccountname.dfs.core.windows.net/geo-location/
"

In [0]:
geoLocationSourceLayerName = 'bronze'
geoLocationSourceStorageAccountName = 'lckudadatalakehousedev'
geoLocationSourceFolderName = 'geo-location'

geoLocationSourceFolderPath = f"abfss://{geoLocationSourceLayerName}@{geoLocationSourceStorageAccountName}.dfs.core.windows.net/{geoLocationSourceFolderName}"

In [0]:
geoLocationBronzeDF = (spark
                       .read
                       .json(geoLocationSourceFolderPath))

In [0]:
display(geoLocationBronzeDF)

In [0]:
from pyspark.sql.functions import *

geoLocationSilverDF = ( geoLocationBronzeDF
                       .select(col('results.admin1').alias('stateName')
                              ,col('results.admin2').alias('districtName')
                               , col('results.country').alias('countryName')
                               , col('results.latitude').alias('latitude')
                               , col('results.longitude').alias('longitude')
                               , col('results.name').alias('marketName')
                               , col('results.population').alias('population')
)
)

display(geoLocationSilverDF)

In [0]:
geoLocationStateTransDF = ( geoLocationSilverDF
 .select(explode("stateName")
 ,monotonically_increasing_id().alias('stateSequenceId')))

In [0]:

geoLocationStateTransDF = ( geoLocationSilverDF
 .select(explode("stateName").alias('stateName')
 ,monotonically_increasing_id().alias('stateSequenceId')))

geoLocationDistrictTransDF = (geoLocationSilverDF
 .select(explode("districtName").alias('districtName')
         ,monotonically_increasing_id().alias('districtSequenceId')
))
geoLocationCountryTransDF = (geoLocationSilverDF
 .select(explode("countryName").alias('countryName')
 ,monotonically_increasing_id().alias('countryNameSequenceId')))

geoLocationLatitudeTransDF = (geoLocationSilverDF
 .select(explode("latitude").alias('latitude')
 ,monotonically_increasing_id().alias('latitudeSequenceId')))

geoLocationLongitudeTransDF = (geoLocationSilverDF
 .select(explode("longitude").alias('longitude')
 ,monotonically_increasing_id().alias('longitudeSequenceId')))

geoLocationMarkeTransDF = (geoLocationSilverDF
 .select(explode("marketName").alias('marketName')
 ,monotonically_increasing_id().alias('marketSequenceId')))

geoLocationPopulationTransDF = (geoLocationSilverDF
 .select(explode("population").alias('population')
 ,monotonically_increasing_id().alias('populationSequenceId')))

In [0]:
geoLocationSilverTransDF = ( geoLocationStateTransDF
                            .join(geoLocationDistrictTransDF, col("stateSequenceId") == col("districtSequenceId"))
                             .join(geoLocationCountryTransDF, col("stateSequenceId") == col("countryNameSequenceId")) 
                            .join(geoLocationLatitudeTransDF, col("stateSequenceId") == col("latitudeSequenceId")) 
                            .join(geoLocationLongitudeTransDF, col("stateSequenceId") == col("longitudeSequenceId")) 
                            .join(geoLocationMarkeTransDF, col("stateSequenceId") == col("marketSequenceId")) 
                            .join(geoLocationPopulationTransDF, col("stateSequenceId") == col("populationSequenceId"))  
                                 .select(col("stateName")
                                    ,col("districtName")
                                  ,col("countryName")
                                    ,col("latitude")
                                    ,col("longitude")
                                    ,col("marketName")
                                    ,col("population")

                             ) )
display(geoLocationSilverTransDF)                             

In [0]:
(geoLocationSilverTransDF 
.write
.mode('overwrite')
.saveAsTable("pricing_analytics.silver.geo_location_silver"))

In [0]:
%sql
select * from pricing_analytics.silver.geo_location_silver geolocation where geolocation.MARkETNAME = 'Guntur'

In [0]:
%sql
SELECT * FROM pricing_analytics.silver.daily_pricing_silver dailyPricing WHERE dailyPricing.MARkET_NAME = 'Guntur'

In [0]:
%sql
SELECT DISTINCT geolocation.*
 FROM pricing_analytics.silver.geo_location_silver geolocation
inner join pricing_analytics.silver.daily_pricing_silver dailyPricing
on geolocation.stateName = dailyPricing.STATE_NAME
and geolocation.marketName = dailyPricing.MARKET_NAME
where geolocation.countryName = 'India'